# sscan in ophyd

Support the [sscan record](https://epics.anl.gov/bcda/synApps/sscan/sscanRecord.html) as an [ophyd](http://nsls-ii.github.io/ophyd) Device for use with data acquisition.  

In [1]:
import time

# common IOC prefix to be used
P = "gp:"

In [2]:
from ophyd.scaler import ScalerCH
scaler = ScalerCH(f"{P}scaler1", name="scaler")
scaler.select_channels(None)

In [3]:
from ophyd import EpicsMotor
m1 = EpicsMotor(f"{P}m1", name="m1")
m2 = EpicsMotor(f"{P}m2", name="m2")

In [4]:
from apstools.synApps import UserCalcsDevice
calcs = UserCalcsDevice(P, name="calcs")

In [5]:
from apstools.synApps import SscanDevice
scans = SscanDevice(P, name="scans")
scans.select_channels()

In [6]:
from apstools.synApps import SaveData
save_data = SaveData(f"{P}saveData_", name="save_data")

In [7]:
# configure a noisy calc, the sscan record will trigger it to process

# clear out the weeds
calcs.reset()

calc = calcs.calc1
calc.channels.A.input_value.put(0.2)
calc.channels.B.input_value.put(0.1)
calc.channels.C.input_pv.put(scans.scan1.current_point.pvname)
calc.calculation.put("A+B*rndm/(C+0.5)")
time.sleep(0.1)          # allow record to react in EPICS IOC
calc.process_record.put(1)

In [8]:
# configure saveData for data collection into MDA files:
        
save_data.file_system.put("/tmp")
save_data.subdirectory.put("saveData")
save_data.base_name.put("sscan1_")
save_data.next_scan_number.put(1)
save_data.comment1.put("testing")
save_data.comment2.put("configured and run from ophyd")

In [9]:
# configure the sscan record for data collection:

# clear out the weeds
scans.reset()

scan = scans.scan1
scan.number_points.put(6)
scan.positioners.p1.setpoint_pv.put(m1.user_setpoint.pvname)
scan.positioners.p1.readback_pv.put(m1.user_readback.pvname)
scan.positioners.p1.start.put(-1)
scan.positioners.p1.end.put(0)
scan.positioner_delay.put(0.0)
scan.detector_delay.put(0.1)
scan.detectors.d01.input_pv.put(scaler.channels.chan03.s.pvname)
scan.detectors.d02.input_pv.put(scaler.channels.chan02.s.pvname)
scan.detectors.d03.input_pv.put(calcs.calc1.calculated_value.pvname)
scan.triggers.t1.trigger_pv.put(scaler.count.pvname)
scan.triggers.t2.trigger_pv.put(calcs.calc1.process_record.pvname)

# finally, reconfigure
scans.select_channels()

In [10]:
# try to run a scan with only ophyd (not bluesky)

row_contents = []
for part in (scans.scan1.positioners, scans.scan1.detectors):
    for chname in part.read_attrs:
        if not chname.endswith("_value"):
            continue
        #if chname.split(".")[-1] not in ("readback_value", "current_value"):
        #    continue
        row_contents.append(getattr(part, chname))

columns = "point time_elapsed".split()
columns += [item.name.lstrip(scans.scan1.name+"_") for item in row_contents]

# start the sscan record
scans.scan1.execute_scan.put(1)
t0 = time.time()
last_point = scans.scan1.current_point.get()

print(*columns)

while True:
    elapsed = time.time() - t0
    current_point = scans.scan1.current_point.get()

    if False:    # diagnostics only
        print(
            "%.3f" % elapsed, 
            scans.scan1.scan_phase.get(as_string=True), 
            scans.scan1.data_state.get(as_string=True), 
            scans.scan1.scan_busy.get(),
            scans.scan1.data_ready.get(),
        )

    if last_point != current_point and current_point > 0:
        row = [
            scans.scan1.current_point.get(),
            "%.3f" % elapsed,             
        ] + [item.get() for item in row_contents]
        print(*row)
    last_point = current_point
    if scans.scan1.scan_phase.get() in (0, "IDLE"):
        break
    time.sleep(0.001)

point time_elapsed positioners_p1_readback_value positioners_p1_setpoint_value detectors_d01_current_value detectors_d02_current_value detectors_d03_current_value
1 2.644 -1.0 -0.8 5.0 3.0 0.28692150115966797
2 4.352 -0.8 -0.6000000000000001 4.0 3.0 0.22903181612491608
3 6.058 -0.6 -0.4000000000000001 2.0 7.0 0.23075242340564728
4 7.758 -0.4 -0.20000000000000007 19.0 15.0 0.22174324095249176
5 9.460 -0.2 -5.551115123125783e-17 6.0 6.0 0.21795207262039185
6 11.162 0.0 -5.551115123125783e-17 4.0 5.0 0.21085527539253235
